In [ ]:
from dotenv import load_dotenv
from groq import Groq

import os

load_dotenv('../.env')


client = Groq(
  api_key=os.getenv("GROQ_API_KEY")
)

MODEL = "llama-3.3-70b-versatile"

# test Groq connection
chat_completion = client.chat.completions.create(
  messages=[
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": "Explain the importance of fast language models",
  }
  ],
  model=MODEL,
  max_completion_tokens=250
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's world of natural language processing (NLP) for several reasons:

1. **Efficient Processing**: Fast language models can process and analyze large amounts of text data quickly, making them ideal for applications that require real-time or near-real-time processing. This is particularly important for tasks such as sentiment analysis, entity recognition, and language translation.
2. **Improved User Experience**: Fast language models enable applications to respond quickly to user input, providing a seamless and interactive experience. This is especially important for chatbots, virtual assistants, and other conversational AI systems.
3. **Scalability**: Fast language models can handle a large volume of requests and process them quickly, making them scalable for large-scale applications. This is critical for industries such as customer service, where a large number of customers may be interacting with a chatbot or virtual assistant simultaneously.
4.

In [57]:
import random
import json

known_weather_data = {
  'berlin': 20.0
}

def get_weather(city: str) -> float:
  city = city.strip().lower()

  if city in known_weather_data:
      return known_weather_data[city]

  return round(random.uniform(-5, 35), 1)

get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Retrieves the temperature for a specified city.",
        "parameters": {
            "type": "object",
            "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city, e.g. New York"
            }
            },
            "required": ["city"],
            "additionalProperties": False
        }
    }
}

# imports calculate function from step 1
def run_conversation(user_prompt):
    # Initialize the conversation with system and user messages
    messages=[
        {
            "role": "system",
            "content": "You are a weather assistant. Use the get weather function to get the weather for a given city and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    # Define the available tools (i.e. functions) for our model to use
    tools = [get_weather_tool]
    # Make the initial API call to Groq
    response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        stream=False,
        tools=tools, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_completion_tokens=4096 # Maximum number of tokens to allow in our response
    )
    print("response", response)
    # Extract the response and any tool call responses
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        # Define the available tools that can be called by the LLM
        available_functions = {
            "get_weather": get_weather,
        }
        # Add the LLM's response to the conversation
        messages.append(response_message)

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the tool and get the response
            function_response = function_to_call(
                city=function_args.get("city")
            )
            function_response = str(function_response)
            # Add the tool response to the conversation
            messages.append(
                {
                    "tool_call_id": tool_call.id, 
                    "role": "tool", # Indicates this message is from tool use
                    "name": function_name,
                    "content": function_response,
                }
            )
        # Make a second API call with the updated conversation
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        print("second_response", second_response)
        # Return the final response
        return second_response.choices[0].message.content
# Example usage
user_prompt = "What is the weather in berlin?"
print(run_conversation(user_prompt))

response ChatCompletion(id='chatcmpl-3c689a49-e9d8-4379-b6d3-7bfab7cb6c1b', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='yhrsr6ktz', function=Function(arguments='{"city":"Berlin"}', name='get_weather'), type='function')]))], created=1752563883, model='llama-3.3-70b-versatile', object='chat.completion', system_fingerprint='fp_6507bcfb6f', usage=CompletionUsage(completion_tokens=14, prompt_tokens=268, total_tokens=282, completion_time=0.032647786, prompt_time=0.021502746, queue_time=0.27360635699999997, total_time=0.054150532), usage_breakdown=None, x_groq={'id': 'req_01k06f56v9ecyb360kfbjegsd6'}, service_tier='on_demand')
second_response ChatCompletion(id='chatcmpl-1a5dc82f-0063-4b84-a3ae-71fe3e5f0680', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=

In [65]:
import importlib

importlib.reload(chat_assistant)
import chat_assistant


tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

developer_prompt = """
You are a weather assistant. Use the get weather function to get the weather for a given city and provide the results.
Use the set weather function to set the weather for a given city.
If not city name is provided, return 'No city name provided'.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client,
    model=MODEL
)

In [62]:
chat.run()

Chat ended.


In [66]:
def set_weather(city: str, temp: float) -> None:
  city = city.strip().lower()
  known_weather_data[city] = temp
  return 'OK'

set_weather_tool = {
  "type": "function",
  "function": {
    "name": "set_weather",
    "description": "Sets the temperature for a specified city.",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "The name of the city, e.g. New York"
        },
        "temp": {
          "type": "number",
          "description": "The temperature to associate with the city, e.g. 20"
        }
      },
      "required": ["city", "temp"],
      "additionalProperties": False
    }
  }
}

tools.add_tool(set_weather, set_weather_tool)

tools.get_tools()

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': 'Retrieves the temperature for a specified city.',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': 'The name of the city, e.g. New York'}},
    'required': ['city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'set_weather',
   'description': 'Sets the temperature for a specified city.',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': 'The name of the city, e.g. New York'},
     'temp': {'type': 'number',
      'description': 'The temperature to associate with the city, e.g. 20'}},
    'required': ['city', 'temp'],
    'additionalProperties': False}}}]

In [67]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client,
    model=MODEL
)
chat.run()

Chat ended.


In [68]:
known_weather_data

{'berlin': 20.0, 'new york': 33}